This notebook generates pharmacophore features from pretrained model (Used for Docking Score prediction - Alpha)

In [1]:
import sys
import os
import torch
from tqdm import tqdm

sys.path.append("..")

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.scoring import PrecalculationModule

In [3]:
MODEL_PATH = '../model_weights/model.tar'
HEAD_PATH = '../model_weights/base_100_per_pocket.pth'

split_file = torch.load('../dataset/pocket_to_ligands.pt')

In [4]:
predictor = PrecalculationModule(MODEL_PATH, HEAD_PATH, 'cuda')

In [5]:
failed_pairs = []
i = 0
for pdb_id, lig_path in tqdm(split_file.items()):
    try:
        lig_path = os.path.join('../dataset/crossdocked_pocket10/', lig_path)
        rec_path = os.path.join('../dataset/crossdock/', pdb_id + '_rec.pdb')
        out_path = rec_path.replace('crossdock', 'docking_pharmacophores').replace('.pdb', '.pt')

        cache = predictor.run(rec_path, ref_ligand_path=lig_path)
        torch.save(cache, out_path)
    except Exception as e:
        failed_pairs.append((rec_path, lig_path))
        print(e)

  0%|          | 35/15307 [00:22<2:26:58,  1.73it/s]

  0%|          | 63/15307 [00:38<2:29:28,  1.70it/s]

  1%|          | 163/15307 [01:46<1:59:49,  2.11it/s]

  1%|▏         | 206/15307 [02:12<2:08:28,  1.96it/s]

  2%|▏         | 265/15307 [02:47<2:19:38,  1.80it/s]

...

 88%|████████▊ | 13468/15307 [2:15:56<11:08,  2.75it/s]

 91%|█████████ | 13937/15307 [2:20:23<13:21,  1.71it/s]

 94%|█████████▍| 14358/15307 [2:24:04<08:20,  1.90it/s]

 96%|█████████▌| 14719/15307 [2:27:15<04:51,  2.02it/s]

 98%|█████████▊| 15077/15307 [2:30:25<02:18,  1.66it/s]==============================
*** Open Babel Warning  in ReadMolecule
There may be erroneous addition spaces or
the file may contains Atom Lists, which are ignored

100%|██████████| 15307/15307 [2:32:29<00:00,  1.67it/s]


Check if all files are saved correctly (not corrupted)

In [14]:
for pdb_id, lig_path in tqdm(split_file.items()):
    lig_path = os.path.join('../dataset/crossdocked_pocket10/', lig_path)
    rec_path = os.path.join('../dataset/crossdock/', pdb_id + '_rec.pdb')
    out_path = rec_path.replace('crossdock', 'docking_pharmacophores').replace('.pdb', '.pt')
    
    if os.path.exists(out_path):
        try:
            torch.load(out_path)
        except:
            cache = predictor.run(rec_path, ref_ligand_path=lig_path)
            print('recomputed', out_path)
            torch.save(cache, out_path)

  4%|▍         | 605/15307 [00:00<00:16, 902.36it/s]

recomputed ../dataset/docking_pharmacophores/3mb6_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5due_A_rec.pt
recomputed ../dataset/docking_pharmacophores/3rvh_A_rec.pt
recomputed ../dataset/docking_pharmacophores/3n86_A_rec.pt
recomputed ../dataset/docking_pharmacophores/2e8a_A_rec.pt
recomputed ../dataset/docking_pharmacophores/2r2w_U_rec.pt
recomputed ../dataset/docking_pharmacophores/4xew_B_rec.pt
recomputed ../dataset/docking_pharmacophores/3aun_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5k01_A_rec.pt
recomputed ../dataset/docking_pharmacophores/1qf6_A_rec.pt
recomputed ../dataset/docking_pharmacophores/1os2_C_rec.pt
recomputed ../dataset/docking_pharmacophores/1akc_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4y6u_A_rec.pt
recomputed ../dataset/docking_pharmacophores/2fr6_B_rec.pt
recomputed ../dataset/docking_pharmacophores/4qx5_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5ap3_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5b0x_A_rec.

  4%|▍         | 628/15307 [00:12<06:55, 35.34it/s] 

recomputed ../dataset/docking_pharmacophores/3o57_A_rec.pt


  4%|▍         | 629/15307 [00:13<07:29, 32.65it/s]

recomputed ../dataset/docking_pharmacophores/3qpn_A_rec.pt
recomputed ../dataset/docking_pharmacophores/1ros_A_rec.pt
recomputed ../dataset/docking_pharmacophores/3nbw_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5j82_A_rec.pt
recomputed ../dataset/docking_pharmacophores/1ogx_B_rec.pt
recomputed ../dataset/docking_pharmacophores/4fv4_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4ymq_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4lm0_A_rec.pt
recomputed ../dataset/docking_pharmacophores/3b65_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4kbn_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4uba_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5mxk_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4elv_A_rec.pt
recomputed ../dataset/docking_pharmacophores/5ly1_D_rec.pt
recomputed ../dataset/docking_pharmacophores/4rse_A_rec.pt
recomputed ../dataset/docking_pharmacophores/1bwb_A_rec.pt
recomputed ../dataset/docking_pharmacophores/4rrx_A_rec.

  6%|▌         | 937/15307 [00:27<07:38, 31.32it/s]

recomputed ../dataset/docking_pharmacophores/4jev_A_rec.pt


100%|██████████| 15307/15307 [00:28<00:00, 530.07it/s]


In [1]:
import torch
pdb_id = '3g1h_A'

torch.load(f"../dataset/docking_pharmacophores/{pdb_id}_rec.pt")

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'pocket_features': tensor([-7.1359e+00, -7.1857e+00,  8.9155e+00, -7.2242e+00,  8.0860e+00,
          1.2627e+01, -1.1418e+01,  3.2766e+01, -1.1102e+01, -9.8266e+00,
         -6.9982e+00, -2.1936e+00, -7.3658e+00, -1.5857e+00,  1.2520e+01,
         -9.0540e-04,  7.2046e+00,  9.9292e+00, -1.0425e+01, -7.2135e+00,
         -8.5731e+00,  8.0012e+00, -9.4897e+00, -9.6983e+00,  8.0898e+00,
         -8.9129e+00,  7.5269e+00,  9.1453e+00, -3.6132e-02,  8.4376e+00,
          1.0082e+01, -8.3576e+00,  7.8823e+00,  7.9983e+00, -1.2288e+01,
         -1.7131e+00, -8.1153e+00,  1.7201e+00, -9.8182e+00, -7.8292e+00,
         -1.0306e+01, -1.2755e+00, -6.1770e+00,  9.2218e+00,  8.8829e+00,
         -9.7712e+00, -8.1676e+00, -1.8682e+00,  1.1439e+01, -6.9962e+00,
         -7.1979e+00, -7.4531e+00,  7.0487e+00, -1.0011e+01,  1.1729e+01,
          1.2459e+01,  1.0086e+01,  5.5723e+00,  2.3753e+01, -3.3125e+00,
         -8.5627e+00, -7.5562e+00,  6.7505e+00, -4.9461e-01,  1.2769e+01,
          7.3774e+0